In [8]:
import os
import math
import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset
from scipy.io import wavfile
import scipy.signal
import numpy as np

In [9]:
NUM_CLASSES = 3
BATCH_SIZE = 20
LEARING_RATE = 0.0001
SONG_LENGTH_SECONDS = 10

In [10]:
class MusicDataset(Dataset):
    def __init__(self, directory, genres, downsample=None):
        self.directory = directory
        self.files = []
        self.downsample = downsample
        for label, genre in enumerate(genres):
            genre_path = os.path.join(directory, genre)
            self.files.extend([(os.path.join(genre_path, f), label) for f in os.listdir(genre_path)])

    def __getitem__(self, index):
        song, label = self.files[index]
        rate, data = wavfile.read(f'{self.directory}/{song}')
        if self.downsample:
            data = scipy.signal.resample(data, self.downsample * SONG_LENGTH_SECONDS)
        tensor = torch.Tensor(data) / (1 << 31)
        return tensor, torch.tensor(label, dtype=torch.long)
    
    def __len__(self):
        return len(self.files)

d = MusicDataset('.', ['rock', 'electro', 'classic'])
train, validate = random_split(d, [900, 300])
input_size = len(train[0][0])

loader = DataLoader(train, batch_size=BATCH_SIZE)
validation_loader = DataLoader(validate, batch_size=BATCH_SIZE)

In [11]:
class Model1Linear(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.h1 = nn.Linear(input_size, hidden_size)
        self.h2 = nn.Linear(hidden_size, hidden_size)
        self.h3 = nn.Linear(hidden_size, hidden_size)
        self.h4 = nn.Linear(hidden_size, hidden_size)
        self.h5 = nn.Linear(hidden_size, hidden_size)
        self.h6 = nn.Linear(hidden_size, hidden_size)
        self.h7 = nn.Linear(hidden_size, hidden_size)
        self.h8 = nn.Linear(hidden_size, hidden_size)
        self.h9 = nn.Linear(hidden_size, NUM_CLASSES)
    
    def forward(self, x):
        x = x.data.view(-1, input_size)
         
        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h4(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h5(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h6(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h7(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h8(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [76]:
def evalulate(model):
    model.eval()
    loss = 0.0
    for data, labels in validation_loader:
        predictions_per_class = model(data.cuda())
        _, highest_prediction_class = predictions_per_class.max(1)
        loss += F.nll_loss(predictions_per_class, labels.cuda())
    return loss/len(validation_loader)

def learn(model, epochs=30):
    torch.cuda.empty_cache()
    optimizer = Adam(params=model.parameters(), lr=LEARING_RATE)

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for data, labels in loader:
            predictions_per_class = model(data.cuda())
            highest_prediction, highest_prediction_class = predictions_per_class.max(1)

            # how good are we? compare output with the target classes
            loss = F.nll_loss(predictions_per_class, labels.cuda())
            total_loss += loss.item()

            model.zero_grad()
            loss.backward()
            optimizer.step()
        
        train_loss = total_loss/len(loader)
        validation_loss = evalulate(model)
        print(f'Epoch: {epoch}, Train Loss: {train_loss}, Validation Loss: {validation_loss.item()}')
        
    return model

In [ ]:
model = Model1Linear(500).cuda()
learn(model, 100)